In [25]:
loader = UnstructuredMarkdownLoader('./testplan_markdown_documentation/o/testplan/doc/en/introduction.md')

In [26]:
data = loader.load()


In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [28]:
print (f'Now you have {len(texts)} documents')


Now you have 16 documents


In [10]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

c:\Dev\workspace\python\Virtual environments\sandbox\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
OPENAI_API_KEY = '...'
PINECONE_API_KEY = '...'
PINECONE_API_ENV = '...'

In [12]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [16]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "testplan"

In [30]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [19]:
query = "What are the supported python versions for testplan?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [20]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [21]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [23]:
query = "How to install testplan?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [33]:
query = 'What are the mandatory parts in test_plan.py? Give code examples'
docs = docsearch.similarity_search(query, include_metadata=True)

In [34]:
chain.run(input_documents=docs, question=query)

" The mandatory parts in test_plan.py are the imports to create the plan object and the tests hierarchy, the piece of code to be tested, and the main function that instantiates a Testplan object and adds tests to it. Code examples are: \n\n``` python\nimport sys\nfrom testplan import test_plan\nfrom testplan.testing.multitest import MultiTest, testsuite, testcase\n\ndef multiply(numA, numB):\n    return numA * numB\n\n@test_plan(name='Multiply')\ndef main(plan):\n    test = MultiTest(name='MultiplyTest',\n                     suites=[BasicSuite()])\n    plan.add(test)\n\nif name == 'main':\n  sys.exit(not main())\n```"

In [42]:
print("The mandatory parts in test_plan.py are the imports to create the plan object and the tests hierarchy,\n the piece of code to be tested, and the main function that instantiates a Testplan object and adds tests to it. Code examples are: \n\n``` python\nimport sys\nfrom testplan import test_plan\nfrom testplan.testing.multitest import MultiTest, testsuite, testcase\n\ndef multiply(numA, numB):\n    return numA * numB\n\n@test_plan(name='Multiply')\ndef main(plan):\n    test = MultiTest(name='MultiplyTest',\n                     suites=[BasicSuite()])\n    plan.add(test)\n\nif name == 'main':\n  sys.exit(not main())\n```")

The mandatory parts in test_plan.py are the imports to create the plan object and the tests hierarchy,
 the piece of code to be tested, and the main function that instantiates a Testplan object and adds tests to it. Code examples are: 

``` python
import sys
from testplan import test_plan
from testplan.testing.multitest import MultiTest, testsuite, testcase

def multiply(numA, numB):
    return numA * numB

@test_plan(name='Multiply')
def main(plan):
    test = MultiTest(name='MultiplyTest',
                     suites=[BasicSuite()])
    plan.add(test)

if name == 'main':
  sys.exit(not main())
```
